# The truncated model 

The intuition of the truncation method can be summed up in two steps.

1. Aggregating together agents with the same recent idiosyncratic history (called *truncated history*).
2. Expressing the model in terms of these groups of agents instead of individual agents. 

This generates the so-called **truncated model**, whose main advantage will be to feature a *finite state space*. 

The rest of this notebook consists in building the truncated model from the Aiyagari model. In terms of Julia objects, we will transform an `AiyagariSolution` (i.e., solution of the Aiyagari model) and a `Economy` (i.e., parameters characterizing the economy) into a `TruncatedModel` object. The details of the construction of the objects can be found in `Structures.ipynb`.

The construction of the truncated model is done in several steps: 
1. Constructing the set of truncated histories
2. lzefj
3. 

In each step, we briefly recall the formal aspects and its connection with the implementation. In particular, we specify how the mathematical objects are computationally represented. 

## Contructing the set of truncated histories


### Contructing *uniform* truncated histories

A truncated history is a vector of given length $N$ gathering idiosyncratic shocks for $N$ periods. For instance, if the idiosyncratic shock is denoted by $y$ taking value in a set $\mathcal Y$,  $h=(\tilde{y}_{-N+1},\ldots,\tilde{y}_{-1},\tilde{y}_{0})\in \mathcal{Y}^N$ is an example of a truncated history of length $N$. An agent $i$ with idiosyncratic history $\{y_{i,0},\ldots,y_{i,t}\}$ in period $t$ will have truncated history $h$ if their history for the last $N$ periods is $h$, i.e., if $(y_{i,t-n+1},\ldots,y_{i,t})=(\tilde{y}_{-N+1},\ldots,\tilde{y}_{-1},\tilde{y}_{0})$.

These truncated histories are called *uniform*, because they all have the same length $N$.

The set of uniform truncated histories is thus simply $\mathcal{Y}^N$. Our implementation features two twists. 
1. Truncated histories will be vectors, but ordered from the current state (first element) to the state $N-1$ periods ago (last element).
2. The vector will not gather the idiosyncratic states themselves, but their corresponding index in the vector representing the set $\mathcal Y$.

Let us give an example. Let assume $\mathcal Y = \{y_1,y_2,y_3\}$, which is represented by the vector `ys = [y1,y2,y3]` (element of the `struct Economy`, see `Structures.ipynb`). Let consider an agent whose idiosyncratic histories is $(\ldots,y_3,y_2,y_2,y_1,y_1)$. This should be read as: the agent is currently in state $y_1$, was one period ago in state $y_1$, was 2 and 3 periods ago in state $y_2$ and 4 periods ago in state $y_3$. 

This will represented by the `Vector{Int}` equal to: `[1,1,2,2,3]` where we assume a truncation length equal to $N=5$ periods (i.e., reversed order and indices in `ys`). 

#### The implementation

The function `generateHistories(N::I,ny::I)::Vector{Vector{I}} where I<:Int` takes as inputs:

* a truncation length `N` (same meaning as $N$);
* a number of idiosyncractic states `ny`(same meaning as $Card(\mathcal Y)$).

Each of the two inputs is requested to be greater than 1. 

The function returns, as a `Vector`, all idiosyncratic histories. Each idiosyncratic history is also a `Vector` (of length `N`). Hence the output is of type `Vector{Vector{I}}` (vector of vectors). By construction the output is of cardinal $Card(\mathcal Y)^N$ or `ny^N`.

The implementation is functional in spirit and takes advantage of the functions `map` and `mapreduce`. The performance is not key since the function is only called once for the model truncation.

***Remark.*** The order of the set of truncated histories does not matter. What matters is that is always iterated in the same order.  

In [1]:
function generateHistories(N::I,ny::I)::Vector{Vector{I}} where I<:Int
    @assert N≥1 && ny≥1
    if N==1
        return map(n->[n],1:ny)
    else
        return mapreduce(h -> map(n->push!(copy(h),n),1:ny),vcat,generateHistories(N-1,ny))
    end
end;

#### Two examples

We check the output for: 

* `N=2` and `ny=2`
* `N=3` and `ny=4`

In [2]:
generateHistories(2,2)

4-element Vector{Vector{Int64}}:
 [1, 1]
 [1, 2]
 [2, 1]
 [2, 2]

In [3]:
generateHistories(3,4)

64-element Vector{Vector{Int64}}:
 [1, 1, 1]
 [1, 1, 2]
 [1, 1, 3]
 [1, 1, 4]
 [1, 2, 1]
 [1, 2, 2]
 [1, 2, 3]
 [1, 2, 4]
 [1, 3, 1]
 [1, 3, 2]
 [1, 3, 3]
 [1, 3, 4]
 [1, 4, 1]
 ⋮
 [4, 2, 1]
 [4, 2, 2]
 [4, 2, 3]
 [4, 2, 4]
 [4, 3, 1]
 [4, 3, 2]
 [4, 3, 3]
 [4, 3, 4]
 [4, 4, 1]
 [4, 4, 2]
 [4, 4, 3]
 [4, 4, 4]

### Constructing *refined* truncated histories

Although simple, uniform truncated histories have the drawback of growing exponentially with $N$, and of including some histories which arevery unlikely to be experienced by agents and hence of very small size.

To solve this issue, an option (see LeGrand and Ragot, AES, 2022) is to consider different truncation lengths for different histories, that will be called *refined* truncated histories. Histories that are more likely to be experienced (i.e., larger ones) can be “refined”, meaning that they can be substituted by a set of histories with higher truncation lengths. 

For instance, the truncated history $(y_{1},y_{1})$ ($N=2$) can be refined into $\{(y,y_{1},y_{1}):y\in\mathcal{Y}\}$, where the group of agents who have been in productivity $y_{1}$ for two consecutive periods is split into $Card(\mathcal{Y})$ truncated histories. 

The function we implement takes advantage of the fact that idiosyncratic states are persistent and that large histories are typically the ones where the agent remains in the same state for $N$ periods (e.g., $(y_{1},\ldots ,y_{1})$ for $N$ periods). The function thus only refines these constant histories. In addition to being connected to data properties, this construction allows obtaining a well-defined partition of the set of idiosyncratic histories (which is not the case with an arbitrary refinement).

The refined truncated is characterized by:
* the common truncation length $N$,
* the refined truncation length $N_y^r$ for each idiosyncratic state $y\in\mathcal Y$.


The implementation uses the same representations as in `generateHistories`: the set of histories is a vector of vectors and each truncated history vector is ordered from the most recent to the latest. The vector of refined truncation lengths will be a vector `[N1,N2,...,Nny]` where for example `N2` is the refinement length of the idiosyncratic realization $y_2$.


Let us give an example. We assume $\mathcal Y = \{y_1,y_2\}$, with $N=2$ and $N^r = [4,3]$. This implies that all histories have a minimal length of 2 and that histories where the agent remain in state $y_1$ (resp. $y_2$) are refined up to length 4 (resp. 3).  The refinement can be thought about in 3 steps:
1. We start with the set of uniform truncated histories ($N=2$), which is: `[[1, 1], [1, 2], [2, 1], [2, 2]]`.
2. We then refine `[1, 1]` and `[2, 2]` once and obtain `[[1, 1, 1], [1, 1, 2], [1, 2], [2, 1], [2, 2, 1], [2, 2, 2]]`. Since $N^r_2=3$, the refinement of state 2 is done
3. We refine `[1, 1, 1]` once more and we obtain `[[1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 2], [1, 2], [2, 1], [2, 2, 1], [2, 2, 2]]`. 

#### The implementation

The implementation uses the same representations as in `generateHistories`: the set of histories is a vector of vectors and each truncated history vector is ordered from the most recent to the latest. We take advantage of multiple dispatch to define two versions of `generateHistories`. 

The function `generateHistories(N::I,refiNs::Vector{I})::Vector{Vector{I}} where I<:Int` takes as inputs:

* a truncation length `N` (same meaning as $N$);
* a vector of refined truncation length for each idiosyncractic state `refiNs`(same meaning as $(N^r_1,\ldots,N^r_{Card(\mathcal Y)})$): `refiNs[k]` gives the truncation length of the `k`th idiosyncratic state.

We request `N ≥ 1` and `length(refiNs) ≥ 1`. If `refiNs[k] ≤ N` for some `k` then no refinement is done but the truncation up to `N` is kept. 

The function returns a vector of vectors of type `Vector{Vector{I}}` as in the uniform case -- with the difference that not all vectors are of the same size. Again, the implementation is functional and the performance is not key. 

The function works as follows:
* it builds the set of uniform truncated histories `generateHistories(N,ny)` where `ny = length(refiNs);
* it then refine histories using the function `refine_hist(refiNs::Vector{I},hs::Vector{Vector{I}},ny::I)::Vector{Vector{I}} where I<:Int`. This function is recursive on the vector `refiNs` and stops when all vector elements are negative. 
* the refinement for one individual history is the function `refine_one_hist(h::Vector{I},ny::I,refiNs::Vector{I})::Vector{Vector{I}} where I<:Int`. If `h` is an history with constant state and if this state needs to be refined (the corresponding element in `refiNs` is strictly positive), then the history `h` is substituted by a set of histories where one elemnt of `1:ny`has been added. 



In [4]:
function refine_one_hist(h::Vector{I},refiNs::Vector{I},ny::I)::Vector{Vector{I}} where I<:Int
    if all(h.==h[1]) && refiNs[h[1]] > zero(I)
        return map(n->push!(copy(h),n),1:ny)
    else
        return [copy(h)]
    end
end


function refine_hist(refiNs::Vector{I},hs::Vector{Vector{I}},ny::I)::Vector{Vector{I}} where I<:Int
    if all(refiNs .<= zero(I))
        return hs
    else
        return refine_hist(refiNs.-1,mapreduce(h -> refine_one_hist(h,refiNs,ny),vcat,hs),ny)
    end
end

function generateHistories(N::I,refiNs::Vector{I})::Vector{Vector{I}} where I<:Int
    ny = length(refiNs)
    @assert ny ≥ one(I) && N ≥ one(I)
    return refine_hist(refiNs .- N, generateHistories(N,ny), ny)
end;

#### Example

We consider the example discussed above: $\mathcal Y = \{y_1,y_2\}$, with $N=2$ and $N^r = [4,3]$.

In [5]:
N = 2
refiNs = [4,3]
generateHistories(N,refiNs)

7-element Vector{Vector{Int64}}:
 [1, 1, 1, 1]
 [1, 1, 1, 2]
 [1, 1, 2]
 [1, 2]
 [2, 1]
 [2, 2, 1]
 [2, 2, 2]

#### Verification 

We use a deprecated function to check that outcomes are similar.

In [6]:
# this is an old deprecated function
function refine_one_hist_old(x::Vector{I},ny::I)::Vector{Vector{I}} where I<:Int
    if all(x.==x[1])
        return map(n->push!(copy(x),n),1:ny)
    else
        return [copy(x)]
    end
end

function refine_hist_old(Nr::I,hs::Vector{Vector{I}},ny::I)::Vector{Vector{I}} where I<:Int
    if Nr<=0
        return hs
    else
        refine_hist_old(Nr-1,mapreduce(x -> refine_one_hist_old(x,ny),vcat,hs),ny)
    end
end

function refinedbuild_histories_old(N::I,Vidio::Vector{I},ny::I)::Vector{Vector{I}} where I<:Int
    
    hs  = generateHistories(N,ny)
    # first one
    Vid = (repeat([1],N)) #  refine hte [1,..1] history
    res_i=convertBasisE10(Vid,ny)
    hfin = refine_hist_old(Vidio[1]-N,[hs[res_i]],ny)
    res_n = res_i  # I store the index of the last refined history
    for i=2:ny # refine other histories
        Vid = (repeat([i],N))
        res_i=convertBasisE10(Vid,ny)
        hfin =  vcat(hfin,hs[res_n+1:res_i-1])
        hr = refine_hist_old(Vidio[i]-N,[hs[res_i]],ny)
        res_n = res_i
        hfin =  vcat(hfin,hr)
    end
    return hfin
end

function areEqual(xs::Vector{Vector{T}},ys::Vector{Vector{T}}) where T<:Number
    length(xs) !== length(ys) && return false
    any(map(length,xs) .!== map(length,ys)) && return false
    return all(map(x -> all(x .== zero(eltype(x))), xs.-ys))
end

areEqual (generic function with 1 method)

In [7]:
@nbinclude("Utils_SW.ipynb");
refiNs = [4,3,3,5,3,2,8]
N = 3

xs =  generateHistories(N,refiNs) 
ys = refinedbuild_histories_old(N,refiNs,length(refiNs))
if areEqual(xs,ys)
    xs
else
    @warn("xs and ys not equal")
end

areEqual(xs, ys) = true


391-element Vector{Vector{Int64}}:
 [1, 1, 1, 1]
 [1, 1, 1, 2]
 [1, 1, 1, 3]
 [1, 1, 1, 4]
 [1, 1, 1, 5]
 [1, 1, 1, 6]
 [1, 1, 1, 7]
 [1, 1, 2]
 [1, 1, 3]
 [1, 1, 4]
 [1, 1, 5]
 [1, 1, 6]
 [1, 1, 7]
 ⋮
 [7, 7, 7, 7, 7, 7, 2]
 [7, 7, 7, 7, 7, 7, 3]
 [7, 7, 7, 7, 7, 7, 4]
 [7, 7, 7, 7, 7, 7, 5]
 [7, 7, 7, 7, 7, 7, 6]
 [7, 7, 7, 7, 7, 7, 7, 1]
 [7, 7, 7, 7, 7, 7, 7, 2]
 [7, 7, 7, 7, 7, 7, 7, 3]
 [7, 7, 7, 7, 7, 7, 7, 4]
 [7, 7, 7, 7, 7, 7, 7, 5]
 [7, 7, 7, 7, 7, 7, 7, 6]
 [7, 7, 7, 7, 7, 7, 7, 7]

## Computing transition probabilities and stationary distributions for truncated histories

We now characterize the distribution of truncated histories. More precisely, we compute:
* the stationary distribution of truncated histories: function `historySizes`;
* the transition matrix over a set of truncated histories: `historyTrans`;

### Stationary distribution with `historySizes`

The function `historySizes(hs::Vector{Vector{I}}, Πy::Matrix{T};
    maxiter::I=1000000, tol::T=1e-16) where{I<:Integer, T<:Real}` takes as inputs:
    
* a set of truncated histories `hs`;
* a transition matrix `Πy` between idiosyncratic histories (`Πy[hs[2],hs[1]]` is the transition probabilty from state `hs[2]` to state `hs[1]` (be careful of the order in `hs`);
* control parameters `maxiter` and `tol` that are used to control when computing the stationary distribution over idiosyncratic states (using `powm!`).


The function returns a tuple `(ind_h, S_h, y0h[ind_h])` with:
* `ind_h` is the index of history with positive sizes: the size of elements `hs[ind_h]` is strictly positive and the one of others is null;
* `S_h` is the size of positive histories (same dimension as `ind_h`)
* `y0h[ind_h]` is the vector current productivity indices for positive size histories. 

***Remarks***
* this function allows one to work only with trauncated histories of non-zero sizes. In practice, this alllows one to diminish the number of histories to consider.
* the size of `ind_h` is not known ex ante. 

In [8]:
function historySizes(hs::Vector{Vector{I}}, Πy::Matrix{T};
    maxiter::I=1000000, tol::T=1e-16) where{I<:Integer, T<:Real}

    # nb of productivity states
    ny = size(Πy,1)       
    
    # computing the stationary distribution over productivity levels
    _,Sy = powm!(Πy', ones(typeof(Πy[1]), ny),maxiter = maxiter,tol = tol)
    Sy  /= sum(Sy)  
    
    # initialization
    Ntot = length(hs)   # total nb of truncated histories
    Sh   = zeros(T, Ntot) 
                        # initialization of size of truncated histories
    y0h   = zeros(I, Ntot) 
                        # initialization of current productivity indices of history

    # We loop over all truncated histories
    nh = zero(I)
    for (ih,h) in enumerate(hs)
        nh = length(h)
        Sh[ih] = Sy[h[nh]] 
                    # distribution according to the 
                    # terminal productity level
        y0h[ih] = h[1]
                    # current prod. level of h
        for j = nh-1:-1:1
            Sh[ih] *= Πy[h[j+1],h[j]] 
            # We move backwards and compute the probability to move 
            # from productivity h[j+1] to productivity h[j]
        end  
    end
    
    # we extract histories with non-zero size
    ind_h, S_h = findnz(sparsevec(Sh))
    return ind_h, S_h, y0h[ind_h]
end;

### Transition matrix between truncated histories with `historyTrans`

The function `historyTrans(hs::Vector{Vector{I}}, Πy::Matrix{T} where{I<:Integer, T<:Real}` takes as inputs (same before):
    
* a set of truncated histories `hs`;
* a transition matrix `Πy` between idiosyncratic histories.

The function returns a tuple transition matrix `Πh` between the elements of `hs`. For instance `Πh[1,2]`is the transition probability between `hs[1]` and `hs[2]`.  

***Remarks***
* we check that the matrix is a proper transition matrix (summing to 1 in rows) using the function `isTransMat`; 
* `hs` must be such that the output is a well-defined transition matrix (it is safe to remove zero-size truncated histories).

In [9]:
function isTransMat(M::Matrix{T})::Bool where{T<:Real}
    all(M.≥zero(T)) || return false             # if one element is negative
    all(M.≤one(T)) || return false              # if one element is greater than 1
    all(sum(M,dims=2) .≈ one(T))|| return false # if one row does not sum to 1
    return true
end

function isTransMat(M::SparseMatrixCSC)
    return isTransMat(Matrix(M))
end

function historyTrans(hs::Vector{Vector{I}},Πy::Matrix{T}) where{I<:Integer,T<:Real}
    # We construct a sparse square matrix Πh of length equal to the one of hs 
    Ntot    = length(hs)
    Πh      = spzeros(T,Ntot,Ntot)
    nh, nht = zero(I), zero(I)
    
    # we loop over histories 
    # we will compute the transition proba from history h
    for (i,h) ∈ enumerate(hs)
        nh = length(h)
        # we loop over destination histories
        # we compute transition proba from h to ht
        for (j,ht) ∈ enumerate(hs)
            nht = length(ht)
            # we check that is a continuation of h 
            
            # case 1: h is longer than ht
            if (nh≥nht) && all(ht[2:end] .== h[1:nht-1])
                Πh[i,j] = Πy[h[1],ht[1]]
            # case 2: ht is longer than h
            elseif (nht>nh) && all(ht[2:(nh+1)] .== h)
                Πh[i,j] = Πy[h[1],ht[1]]
            end
        end
    end
    @assert isTransMat(Πh) # checking that the matrix is actually a transition matrix.
    return Πh
end;

### Distribution of truncated histories over the asset × productivity grid with `historyDist`

The function `historyDist(hs::Vector{Vector{I}},stationaryDist::Matrix{T}, 
    transitMat::SparseMatrixCSC{T,I},ny::I)::Matrix{T} where{I<:Integer, T<:Real}` takes as inputs:
    
* a set of truncated histories `hs`;
* a stationary distribution `stationaryDist` over the product grid productivity × asset;
* a transition matrix `transitMat` between elements of the product grid productivity × asset;
* the number of productivity levels `ny`.  

The function returns a matrix $((n_a \times n_y), N_{tot})$, where $N_{tot}$ is the length of `hs`, $n_a$ of the asset grid and $n_y$ of the productivity grid). The result matrix contains the distribution of truncated histories over the product grid asset $\times$ productivity. More precisely, if `statDist_h = historyDist(has, stationaryDist, transitMat, ny)`, then  `statDist_h[:,h]` is the distribution of truncated history `h` over the product grid asset $\times$ productivity.

***Remark:*** the stationary distribution and the transition matrix come from the resolution of the Aiyagari model (see [Aiyagari notebook](./SolveAiyagari.ipynb)).

In [10]:
function historyDist(hs::Vector{Vector{I}},stationaryDist::Matrix{T}, 
    transitMat::SparseMatrixCSC{T,I},ny::I)::Matrix{T} where{I<:Integer, T<:Real}

    na   = div(size(transitMat,1),ny) # size of asset grid 
    Ntot = length(hs)                 # nb of truncated histories
    
    # initialization of the stat distribution: matrix of size (na*ny, Ntot)
    statDist_h = zeros(T, na*ny, Ntot)
    temp_v = zeros(T, na)
    nh = zero(I)
    
    # Loop over all truncated histories
    for (ih,h) in enumerate(hs)
        shift0 = (h[end]-1)*na # shift for the index of h[end] in the product grid
        # we initialize with the stationary distribution. 
        statDist_h[1+shift0:na+shift0,ih] =  stationaryDist[:,h[end]]
        nh = length(h)
        for j=nh-1:-1:1
            # shifts for accounting for the asset grid
            shift1 = (h[j+1]-1)*na
            shift0 = (h[j]-1)*na
            
            # we update the stationary distribution using the transition matrix
            temp_v .= statDist_h[1+shift1:na+shift1,ih]
            statDist_h[:,ih] .= zero(T)
            statDist_h[1+shift0:na+shift0,ih] .= (
                transitMat[1+shift0:na+shift0,1+shift1:na+shift1] * temp_v)
        end
    end
    return statDist_h
end;

LoadError: UndefVarError: SparseMatrixCSC not defined

## Credit constrained histories

The function `credit_constrained_h(shareCC::T, S_h::Vector{T}, x_h::Vector{T}; method::String="closest", rev::Bool=true)` identifies credit-constraint histories. Loosely speaking, it works as follows: 
* the histories are sorted along `x_h` in the order specified by `rev` (typically from largest to smallest for Lagrange multipliers and from smallest to largest for wealth);
* the algorithm then selects the first sorted histories such that their cumulative share equals the target share `shareCC`;
* exactly targeting `shareCC` is not possible: either we target the closest one (`method="closest"`) or the first index larger than the target (any other value for `method`).

The function takes as input:
* `shareCC`: the target share of credit-constrained agents;
* `S_h`: the distribution of truncated histories;
* `x_h`: a vector of the same size as `S_h` along which the sorting is done;
* `method`: a string characterizing the selection mechanism, set by default to `"closest"`;
* `rev`: a Boolean (by default true) to specify the order of the sorting (true implies decreasing order).
        
The function returns a pair:
* an integer `i_cc` indicating the number of credit-constrained truncated histories;
* the vector of indices of credit-constrained truncated histories.

***Remarks.***

1. The credit-constrained histories are computed such that based the first `i_cc` truncated histories (ordered along `x_h` in the order `rev`) have a cumylative size, computed using `S_h`, similar to the target `shareCC`. The meaning of *similar* is specified by the method: either the first larger or the closest.

2. This function is key for the accuracy of the simulation. 

In [11]:
function credit_constrained_h(shareCC::T, S_h::Vector{T}, x_h::Vector{T}; 
                method::String="closest",rev::Bool=true) where{T<:Real}
    # We sort S_h along x_h following the order rev
    ind_x_sort = sortperm(x_h,rev=rev)
    S_h_sorted = S_h[ind_x_sort]
    
    if method=="closest"
        i_cc = argmin(abs.(cumsum(S_h_sorted) .- shareCC)) #closest distance
    else 
        i_cc = findlast(x->x<zero(T), cumsum(S_h_sorted) .- shareCC)#at least as large
    end
    i_c = isnothing(i_cc) ? 1 : i_cc # we return at least one credit-constrained history

    return i_c, ind_x_sort[1:i_c]
end;

## Truncating a policy function

The function `truncate_polfun(gx::Matrix{T}, statDist_h::Matrix{T}; f=identity)::Vector{T} where{T<:Real}` computes for the individual policy function `gx` and for the distribution `statDist_h` over truncated histories the aggregation of the variable `x` for all truncated histories. More formally, if we denote by $x_{i,j}$ and $p_{i,j,h}$ the policy function and distribution (where $i$ is the index for the asset grid, $j$ for the productivity grid, and $h$ for histories), then the function computes for all $h$:
$$x_h = \frac{\sum_{i,j}p_{i,j,h}\,x_{i,j}}{\sum_{i,j} p_{i,j,h}}.$$

The function takes as inputs:
* a policy function `gx` which is a matrix of dimension $(n_a,n_y)$;
* a distribution of truncated hsitories over the product grid asset $\times$ productivity. This a matrix of dimension $(n_a \times n_y, N_{tot})$, where $N_{tot}$ is the number of truncated histories.
* a function `f` with which we can transform `gx` (we then compute the aggregation of `f(gx)` which generally differs from `f(xh)`.

The function returns a `Vector` of length  $N_{tot}$ equal to $(x_h)_{h\in\mathcal{H}}$.

In [12]:
function truncate_polfun(gx::Matrix{T}, statDist_h::Matrix{T}; f=identity) where{T<:Real}
    na,ny  = size(gx)
    n,Ntot = size(statDist_h)
    @assert n==na*ny
    p_ijk = reshape(statDist_h,na,ny,Ntot) 
    xhs = zeros(T, Ntot)
    for k in 1:Ntot
        p_j = sum(p_ijk[:,:,k],dims=1)
        x_j = sum(p_ijk[:,:,k].*gx,dims=1)
        f_x_j = zeros(T, size(x_j))
#         @show x_j.!=zero(T)
#         @show x_j[(x_j.!=zero(T))]
#         @show (sum(p_ijk[:,:,k],dims=1)[(x_j.!=zero(T))])
#         @show f.(x_j[(x_j.!=zero(T))])./ (sum(p_ijk[:,:,k],dims=1)[(x_j.!=zero(T))])
        f_x_j[x_j.!=zero(T)] = f.(x_j[x_j.!=zero(T)]./(sum(p_ijk[:,:,k],dims=1)[x_j.!=zero(T)]))
        xhs[k] = (sum(p_j.*f_x_j,dims=2)./sum(p_j,dims=2))[1,1]
    end
    return xhs
end;


truncate_polfun (generic function with 1 method)

## Computing the truncated model

This is the core function of the section that computes the truncated model.

The function `TruncatedModel(N::Integer,refiNs::Vector{Int64}, # length of the truncation
                             solution::AiyagariSolution, economy::Economy;
                             maxiter=1000000,tol=1e-16)` takes as inputs:
* truncation length parameters: the common truncation length `N` and the vector of truncation length `refiNs` for each productivity state;
* a solution of the Aiyagari model,  `solution`;
* an economy characterized by `economy`;
* convergence control parameters `maxiter` and `tol` for the computation of the stationary distribution.

The function returns a `TruncatedModel` (actually, the function can be seen as a specific constructor). See [Structures.ipynb](./Structures.ipynb) for more details about this structure.

In [13]:
function TruncatedModel(N::Integer,refiNs::Vector{Int64}, # length of the truncation
                             solution::AiyagariSolution, economy::Economy;
                             maxiter=1000000,tol=1e-16)

    @unpack β,α,δ,u,u′,u′′,v,v′,na,a_min,aGrid,ny,ys,Πy = economy
    @unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution
    T = typeof(β)
    τ = zero(β)

    # Identifying positive-size truncated histories
    hs′ = generateHistories(N,refiNs)
    (ind_h, S_h, ind_y0_h)= historySizes(hs′,Πy)
    
    # We consider only non-zero histories.
    hs   = hs′[ind_h] # Important : I redefine the histories here to consider the relevant one
    y0_h = [ys[i] for i in ind_y0_h]

    # Transition and distribution of truncated historoes
    statDist_h = historyDist(hs, stationaryDist,         # distribution
                    transitMat, ny) 
    Ntot       = length(hs)                              # nb of truncated histories with positive size
    Π_h        = historyTrans(hs,Πy)              # transition matrix

    # Computing truncated allocations
    c_h        = truncate_polfun(gc, statDist_h)         # consumption
    a_beg_h    = truncate_polfun(repeat(aGrid,1,ny),  # beginning-of-period wealth
                                statDist_h)
    a_end_h    = truncate_polfun(ga, statDist_h)         # end-of-period wealth
    
#     u_h       = truncate_polfun(gc, statDist_h, f=u)     # utility of consumption
    u_h       = truncate_polfun(u.(gc), statDist_h)     # utility of consumption
#     u′_h      = truncate_polfun(gc, statDist_h, f=u′)    # maginal utility of consumption
    u′_h      = truncate_polfun(u′.(gc), statDist_h)    # maginal utility of consumption
    u′′_h     = truncate_polfun(gc, statDist_h, f=u′′)   # 2nd derivative of utility of consumption
    
    v_h        = truncate_polfun(gl, statDist_h, f=v)    # disutility of labor
    v′_h       = truncate_polfun(gl, statDist_h, f=v′)   # marginal disutility of labor
    l_h        = truncate_polfun(gl, statDist_h)         # labor supply
    ly_τ_h     = truncate_polfun((repeat(ys',na,1).*gl).^(1-τ), # efficient labor supply (y×l)^(1-τ)
                                    statDist_h, f=x->x) 
    
    resid_E_h = truncate_polfun(residEuler, statDist_h)  # Euler equation residuals

    # Handling credit-constraints: closest methods based on residuals of the Euler equation
    share_cc = sum(stationaryDist[1,:])                  # Share of credit-constrained agents
    nb_cc_h, ind_cc_h = credit_constrained_h(share_cc,   # Credit-constraint histories
               S_h,resid_E_h,method="closest",rev=true)


    # Constructing the truncated allocation
    truncatedAllocation = TruncatedAllocation(
        S_h       = S_h,
        Π_h       = Π_h,
        y0_h      = y0_h,
        a_beg_h   = a_beg_h,
        a_end_h   = a_end_h,
        c_h       = c_h,
        l_h       = l_h,
        ly_τ_h    = ly_τ_h,
        u_h       = u_h,
        u′_h      = u′_h,
        u′′_h     = u′′_h,
        v_h       = v_h,
        v′_h      = v′_h,
        resid_E_h = resid_E_h,
        nb_cc_h   = nb_cc_h,
        ind_cc_h  = ind_cc_h)

    # Computing the ξs    
    ξu0 = u_h./u.(c_h)
    ξu1 = u′_h./u′.(c_h)
    ξu2 = u′′_h./u′′.(c_h)
    
#     @show u′.(c_h)
#     @show resid_E_h
#     @show Matrix((I-β*R*Π_h)[1:5,1:5])
    ξuE = ((I-β*R*Π_h)\resid_E_h)./u′.(c_h)
    ξy  = truncate_polfun(repeat(ys',na,1).*u′.(gc),statDist_h)./(y0_h.*u′.(c_h))
    ξv0 = v_h./v.(l_h)
    ξv1 = (1-τ)*w*ly_τ_h.*u′_h./(l_h.*v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)

    ξs = ξs_struct(
        ξu0 = ξu0,
        ξu1 = ξu1,
        ξu2 = ξu2,
        ξuE = ξuE,
        ξy  = ξy,
        ξv0 = ξv0,
        ξv1 = ξv1)
    
    return TruncatedModel(
            N                = N,      # common histories
            refiNs           = refiNs, # refined histories
            Ntot             = Ntot,   
            ind_h            = ind_h,
            truncatedAllocation = truncatedAllocation,
            ξs               = ξs)
    
end;

In [ ]:

function test(N::Integer, # length of the truncation
    solution::AiyagariSolution, 
    economy::Economy;maxiter=1000000,tol=1e-16)
@unpack β,α,δ,u,u′,u′′,v,v′,l_supply,na,a_min,aGrid,ny,ys,Πy = economy
@unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution
(ind_h, S_h, ind_y0_h) = historySizes(N, Πy)
y0_h = [ys[i] for i in ind_y0_h]
T = typeof(Πy[1,1])
statDist_h = historyDist(ny, N, stationaryDist, transitMat)[:,ind_h]
Ntot = length(ind_h)#nb of history with positive size
Π_h = historyTrans(N,ind_h,Πy)

#Compute allocations
c_h       = sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h   
l_h       = sum(statDist_h .* repeat(gl[:],1,Ntot), dims=1)[:]./S_h
a_beg_h   = sum(statDist_h .* repeat(aGrid[:],ny,Ntot), dims=1)[:]./S_h
a_end_h   = sum(statDist_h .* repeat(ga[:],1,Ntot), dims=1)[:]./S_h
u_h       = sum(statDist_h .* repeat(u.(gc[:]),1,Ntot), dims=1)[:]./S_h
u′_h      = sum(statDist_h .* repeat(u′.(gc[:]),1,Ntot), dims=1)[:]./S_h
u′′_h     = sum(statDist_h .* repeat(u′′.(gc[:]),1,Ntot), dims=1)[:]./S_h
v_h       = sum(statDist_h .* repeat(v.(gl[:]),1,Ntot), dims=1)[:]./S_h
v′_h      = sum(statDist_h .* repeat(v′.(gl[:]),1,Ntot), dims=1)[:]./S_h
# ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]).^τ,1,Ntot), dims=1)[:]./S_h
ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]),1,Ntot), dims=1)[:]./S_h
resid_E_h = sum(statDist_h .* repeat(residEuler[:],1,Ntot), dims=1)[:]./S_h

#Define credit constrained histories
share_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents
nb_cc_h, ind_cc_h = credit_constrained_h(share_cc, S_h, c_h;method="first larger")

allocation_proj = Allocation_proj(
S_h=S_h,
Π_h=Π_h,
y0_h=y0_h,
a_beg_h=a_beg_h,
a_end_h=a_end_h,
c_h=c_h,
l_h=l_h,
ly_h=ly_h,
u_h=u_h,
u′_h=u′_h,
u′′_h=u′′_h,
v_h=v_h,
v′_h=v′_h,
resid_E_h=resid_E_h,
nb_cc_h=nb_cc_h,
ind_cc_h=ind_cc_h)



return resid_E_h

end



